### 1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-pinecone langchainhub

/Users/lhj/inflearn-langchain-rag/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


### 2. Knowledge Base 구성을 위한 데이터 생성
- split 된 데이터 chunk를 LLM에게 전달하면 토큰 절약 가능 -> 비용 감소와 답변 생성시간 감소의 효과
- LangChain에서 다양한 TextSplitter들을 제공
- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
    - chunk_size 는 split 된 chunk의 최대 크기
    - chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

#### * 이때, 임베딩이 잘 되기 위해서는 데이터 전처리가 필요하다.
- 데이터의 이미지를 텍스트로 변환해야하며
- 표 데이터인 경우 **마크다운**으로 변환해야한다. 

In [ ]:
# text_splitter로 chunking 하는 과정 
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./document/tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
# 임베딩 객체 생성 
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
# 벡터 DB 객체 생성 
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'index-2'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
# 인덱스 새로 만드는 경우 
database = PineconeVectorStore.from_documents(
    document_list, embedding, 
    index_name=index_name
)

In [ ]:
# 기존 인덱스 연결하는 경우 (새로 만들지 않음)
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding  # embedding 객체는 필요
)

### 3. 관련 정보를 가져오는 Retrieval (검색)
- Chroma에 저장한 데이터에서 질문 쿼리와 관련있는 데이터를 가져오는 과정
- similarity_search(): 유사도 검색 제공 
- `k` 값을 조절해서 얼마나 많은 연관 데이터를 불러올지 결정

### * 쿼리를 올바르게 변환해 주는 것이 답변 정확도의 핵심 
- 질문에 있는 '직장인' 이라는 단어와 데이터에 있는 '거주자'라는 단어를 의미적으로 연결하지 못함 
- 키워드 사진을 통해 쿼리를 적절히 변환 했을 때 답변 정확도가 올라감

In [57]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# query = '연봉 5천만원인 거주자의 소득세는 얼마인가요?' -> 답변 정확도 올라감 

from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]
query_transform_prompt = ChatPromptTemplate.from_template(
    """사용자의 질문을 보고, 키워드 사전을 참고해서 사용자의 질문을 변경해주세요. 
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다. 
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    사용자의 질문: {question}
    """
)

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

dic_prompt = query_transform_prompt.invoke({"question": query, "dictionary": dictionary})
ai_response = llm.invoke(dic_prompt)

new_query = ai_response.content

In [53]:
retrieved_docs = database.similarity_search(new_query, k=3)

retrieved_docs

[Document(id='188f2c0f-4d8f-473e-8fc3-cb6842e8d044', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

### 4. Augmentation(증강)을 위한 Prompt 활용
- Retrieval된 데이터를 프롬프트를 활용하여 LLM에 전달 
- LangChain에서 제공하는 프롬포트("rlm/rag-prompt")를 사용해도 되고, 본인이 직접 작성해도 된다. 

In [54]:

context = "\n\n---".join([doc.page_content for doc in retrieved_docs])

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 최고의 한국 소득세 전문가입니다. 
                주어진 context를 기반으로 질문에 답변하세요."""
    ),
    ("human", """Context: {context}
                Question: {question}"""
    )
])
chat_value = rag_prompt.invoke({"context": context, "question": new_query})


### 5. 답변 생성 (Generation)
- 프롬포트를 LLM에게 전달하여 답변 생성 

In [55]:
# 7. LLM 호출
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

ai_response = llm.invoke(chat_value)


In [56]:
ai_response.content

'연봉 5천만원인 거주자의 소득세를 계산해보겠습니다.\n\n1. **과세표준**: 5천만원\n2. **세율 적용**: 5천만원은 "1,400만원 초과 5,000만원 이하" 구간에 해당합니다.\n   - 이 구간은 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)로 계산됩니다.\n\n3. **계산**:\n   - 5,000만원 - 1,400만원 = 3,600만원\n   - 3,600만원의 15% = 540만원\n   - 따라서 소득세 = 84만원 + 540만원 = 624만원\n\n연봉 5천만원인 거주자의 소득세는 **624만원**입니다.'